In [1]:
import modin.pandas as pd
import nums
from nums import numpy as nps

from nums.experimental.nums_modin import from_modin

import warnings
warnings.filterwarnings("ignore");

nums.init()

Automatically increasing RLIMIT_NOFILE to max value of 9223372036854775807


2021-11-02 14:48:43,084	INFO services.py:1265 -- View the Ray dashboard at http://127.0.0.1:8265


Using driver node ip as head node.
head node 10.142.66.97
total cpus 6.0
device_grid (0, 0) 0=node:10.142.66.97/cpu:1


In [2]:
# Set this parameter to the number of years you want to scale the dataset to
years = 1

In [3]:
%%time
X_data = pd.DataFrame()
y_data = pd.DataFrame()

for year in range(2021 - years, 2021):
    X_data = X_data.append(pd.read_csv("data/X_" + str(year) + ".csv.gz", compression='gzip'))
    y_data = y_data.append(pd.read_csv("data/y_bin_" + str(year) + ".csv.gz", compression='gzip'))

CPU times: user 2.72 s, sys: 1.38 s, total: 4.1 s
Wall time: 5.14 s


In [4]:
print(X_data.shape)
print(y_data.shape)

(3272224, 8)
(3272224, 1)


In [5]:
X = from_modin(X_data).astype(float)
y = from_modin(y_data).astype(float).reshape(-1)

In [6]:
shuffle = nps.random.permutation(X.shape[0])
X = X[shuffle]
y = y[shuffle]
split = int(X.shape[0] * 0.8)
X_train = X[:split]
X_test = X[split:]
y_train = y[:split]
y_test = y[split:]

In [7]:
from nums.models.glms import LogisticRegression

model = LogisticRegression(solver="newton", penalty="l2", tol=1e-4)

Here are the times that it takes to fit on NumS' Logistic Regression on a laptop with 8 cores.

| Years       | Number of Data Entries (Rows) | Fitting Time |
| ----------- | ----------- | -------- |
| 1   | 3272224  | 3.74 s | 
| 2   | 6755037  | 5.61 s |
| 3   | 10371000 | 9.16 s | 

In [8]:
%%time
model.fit(X_train, y_train)

CPU times: user 1.66 s, sys: 875 ms, total: 2.53 s
Wall time: 3.74 s


In [9]:
%%time
training_results = model.predict(X_train)

CPU times: user 29.1 ms, sys: 12.1 ms, total: 41.2 ms
Wall time: 23.4 ms


In [10]:
%%time
test_results = model.predict(X_test)

CPU times: user 23.7 ms, sys: 8.35 ms, total: 32 ms
Wall time: 19.9 ms


In [11]:
print("Training accuracy: ", nps.sum(training_results == y_train).get() / y_train.shape[0])
print("Testing accuracy:  ", nps.sum(test_results == y_test).get() / y_test.shape[0])

Training accuracy:  0.7212950367467995
Testing accuracy:   0.721522817043449


In [12]:
X_train_np = X_train.get()
X_test_np = X_test.get()
y_train_np = y_train.get()
y_test_np = y_test.get()

In [13]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

Here are the times that it takes to fit on Sklearn's Logistic Regression on the same machine.

| Years       | Number of Data Entries (Rows) | Fitting Time |
| ----------- | ----------- | -------- |
| 1  | 3272224  | 1min 7s  | 
| 2  | 6755037  | 2min 22s |
| 3  | 10371000 | 3min 43s | 

In [14]:
%%time
model.fit(X_train_np, y_train_np)

CPU times: user 3min 14s, sys: 31.4 s, total: 3min 45s
Wall time: 1min 6s


LogisticRegression()

In [15]:
%%time
training_results = model.predict(X_train_np)

CPU times: user 805 ms, sys: 115 ms, total: 920 ms
Wall time: 121 ms


In [16]:
%%time
test_results = model.predict(X_test_np)

CPU times: user 189 ms, sys: 47 ms, total: 236 ms
Wall time: 29.9 ms


In [17]:
print("Training accuracy: ", model.score(X_train_np, y_train_np))
print("Testing accuracy:  ", model.score(X_test_np, y_test_np))

Training accuracy:  0.7130487332964318
Testing accuracy:   0.7132471025067042
